In [1]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.circuit.library import Permutation, EfficientSU2
from qiskit.quantum_info import random_clifford
from qiskit.transpiler import PassManager, generate_preset_pass_manager

from qiskit_ibm_transpiler.ai.collection import CollectLinearFunctions, CollectCliffords, CollectPauliNetworks, CollectPermutations
from qiskit_ibm_transpiler.ai.synthesis import AILinearFunctionSynthesis, AIPauliNetworkSynthesis, AICliffordSynthesis, AIPermutationSynthesis

In [2]:
from qiskit_ibm_runtime.fake_provider.backends.marrakesh import FakeMarrakesh

backend = FakeMarrakesh()
coupling_map = backend.coupling_map
n_qubits = backend.num_qubits

print(backend.name, backend.num_qubits)

fake_marrakesh 156


## Linear Functions Circuit

In [3]:
iterations = 32
commutative = True
backwards = False

In [4]:
circuit = QuantumCircuit(n_qubits)

for c in range(iterations):
    nq = 8
    qs = np.random.choice(range(circuit.num_qubits), size=nq, replace=False)
    circuit.compose(random_clifford(nq).to_circuit(), qubits=qs.tolist(), inplace=True)
    for q in qs:
        circuit.t(q)

print(
    f"Original Linear Fnc circuit -> Depth: {circuit.decompose(reps=3).depth()}, Gates(2q): {circuit.decompose(reps=3).num_nonlocal_gates()}"
)
#circuit.draw(fold=-1, scale=0.3, style="iqp")

Original Linear Fnc circuit -> Depth: 509, Gates(2q): 1183


In [5]:
qiskit_lvl3_transpiler = generate_preset_pass_manager(
    optimization_level=3, coupling_map=coupling_map
)
lvl3_transpiled_circuit = qiskit_lvl3_transpiler.run(circuit)

In [6]:
import time
start_time = time.time()
ai_linear_functions_synthesis_pass = PassManager(
    [
        CollectLinearFunctions(
            do_commutative_analysis=commutative,
            collect_from_back=backwards,
        ),
        AILinearFunctionSynthesis(coupling_map=coupling_map),
    ]
)

synthesized_circuit_local = ai_linear_functions_synthesis_pass.run(
    lvl3_transpiled_circuit
)
end_time = time.time()
delta = end_time - start_time

In [7]:
print(f"Time taken for local AI synthesis: {delta:.2f} seconds")
print(
    f"Original Linear Fnc circuit -> Depth: {circuit.decompose(reps=3).depth()}, Gates(2q): {circuit.decompose(reps=3).num_nonlocal_gates()}"
)
print(
    f"Synthesized Linear Fnc circuit (local AI) -> Depth: {synthesized_circuit_local.decompose(reps=3).depth()}, Gates(2q): {synthesized_circuit_local.decompose(reps=3).num_nonlocal_gates()}"
)
complexity = circuit.decompose(reps=3).depth() * circuit.decompose(reps=3).num_nonlocal_gates()
speed = circuit.decompose(reps=3).num_nonlocal_gates() / delta
speed2 = complexity / delta
print(f">>>>Gate Speed: {speed:.2f} gates/second")
print(f">>>>Complexity Speed: {speed2:.2f} complexity/second")
#synthesized_circuit_local.draw(fold=-1, scale=0.3, style="iqp")

Time taken for local AI synthesis: 4.06 seconds
Original Linear Fnc circuit -> Depth: 509, Gates(2q): 1183
Synthesized Linear Fnc circuit (local AI) -> Depth: 2186, Gates(2q): 6689
>>>>Gate Speed: 291.48 gates/second
>>>>Complexity Speed: 148361.62 complexity/second


## Permutation Circuit

In [8]:
circuit = QuantumCircuit(n_qubits)
circuit.append(
    Permutation(
        num_qubits=n_qubits, pattern=[(i + 1) % n_qubits for i in range(n_qubits)]
    ),
    qargs=range(n_qubits),
)
circuit = circuit.decompose(reps=2)

print(
    f"Original Permutations circuit -> Depth: {circuit.decompose(reps=3).depth()}, Gates(2q): {circuit.decompose(reps=3).num_nonlocal_gates()}"
)

#circuit.draw(fold=-1, scale=0.3, style="iqp")

Original Permutations circuit -> Depth: 465, Gates(2q): 465


In [9]:
qiskit_lvl3_transpiler = generate_preset_pass_manager(
optimization_level=3, coupling_map=coupling_map
)
lvl3_transpiled_circuit = qiskit_lvl3_transpiler.run(circuit)

In [10]:
import time
start_time = time.time()
ai_linear_functions_synthesis_pass = PassManager(
    [
        CollectPermutations(
            do_commutative_analysis=commutative,
            collect_from_back=backwards,
            max_block_size=n_qubits,
        ), ## or max_block_size=27
        AIPermutationSynthesis(coupling_map=coupling_map),
    ]
)

synthesized_circuit_local = ai_linear_functions_synthesis_pass.run(
    lvl3_transpiled_circuit
)
end_time = time.time()
delta = end_time - start_time

In [11]:
print(f"Time taken for Permutations with Local AI: {delta:.2f} seconds")
print(
    f"Original Permutations circuit -> Depth: {circuit.decompose(reps=3).depth()}, Gates(2q): {circuit.decompose(reps=3).num_nonlocal_gates()}"
)
print(
    f"Synthesized Permutations circuit (local AI) -> Depth: {synthesized_circuit_local.decompose(reps=3).depth()}, Gates(2q): {synthesized_circuit_local.decompose(reps=3).num_nonlocal_gates()}"
)
complexity = circuit.decompose(reps=3).depth() * circuit.decompose(reps=3).num_nonlocal_gates()
speed = circuit.decompose(reps=3).num_nonlocal_gates() / delta
speed2 = complexity / delta
print(f">>>>Gate Speed: {speed:.2f} gates/second")
print(f">>>>Complexity Speed: {speed2:.2f} complexity/second")
#synthesized_circuit_local.draw(fold=-1, scale=0.3, style="iqp")

Time taken for Permutations with Local AI: 0.00 seconds
Original Permutations circuit -> Depth: 465, Gates(2q): 465
Synthesized Permutations circuit (local AI) -> Depth: 0, Gates(2q): 0
>>>>Gate Speed: 154434.35 gates/second
>>>>Complexity Speed: 71811971.05 complexity/second


## Eficient SU2

In [12]:
circuit = EfficientSU2(n_qubits, entanglement="circular", reps=1).decompose()
print(
    f"Original EfficientSU2 circuit -> Depth: {circuit.depth()}, Gates(2q): {circuit.num_nonlocal_gates()}"
)
#circuit.draw(fold=-1, scale=0.2, style="iqp")

Original EfficientSU2 circuit -> Depth: 160, Gates(2q): 156


In [13]:
qiskit_lvl3_transpiler = generate_preset_pass_manager(
optimization_level=3, coupling_map=coupling_map
)
lvl3_transpiled_circuit = qiskit_lvl3_transpiler.run(circuit)

In [14]:
import time
start_time = time.time()
ai_linear_functions_synthesis_pass = PassManager(
    [
        CollectLinearFunctions(
            do_commutative_analysis=commutative,
            collect_from_back=backwards,
        ),
        AILinearFunctionSynthesis(coupling_map=coupling_map),
    ]
)

synthesized_circuit_local = ai_linear_functions_synthesis_pass.run(
    lvl3_transpiled_circuit
)
end_time = time.time()
delta = end_time - start_time

In [15]:
print(f"Time taken for EfficientSU2 with Local AI: {delta:.2f} seconds")
print(
    f"Original EfficientSU2 circuit -> Depth: {circuit.decompose(reps=3).depth()}, Gates(2q): {circuit.decompose(reps=3).num_nonlocal_gates()}"
)
print(
    f"Synthesized EfficientSU2 circuit (local AI) -> Depth: {synthesized_circuit_local.decompose(reps=3).depth()}, Gates(2q): {synthesized_circuit_local.decompose(reps=3).num_nonlocal_gates()}"
)
complexity = circuit.decompose(reps=3).depth() * circuit.decompose(reps=3).num_nonlocal_gates()
speed = circuit.decompose(reps=3).num_nonlocal_gates() / delta
speed2 = complexity / delta
print(f">>>>Gate Speed: {speed:.2f} gates/second")
print(f">>>>Complexity Speed: {speed2:.2f} complexity/second")
#synthesized_circuit_local.draw(fold=-1, scale=0.3, style="iqp")

Time taken for EfficientSU2 with Local AI: 0.48 seconds
Original EfficientSU2 circuit -> Depth: 160, Gates(2q): 156
Synthesized EfficientSU2 circuit (local AI) -> Depth: 588, Gates(2q): 591
>>>>Gate Speed: 327.02 gates/second
>>>>Complexity Speed: 52322.83 complexity/second
